# Laboratorio 6
Francisco Castillo - 21562

Diego Lemus - 21

## Task 1
### ¿Qué es Prioritized Sweeping para ambientes determinísticos?
Es una técnica de planeamiento que permite acelerar la propagación de los valores de los estados sin necesidad de actualizarlos todos uniformemente. En lugar de recorrer cada estado del espacio, se utiliza una cola de prioridades en la que se ordenan los estados según la magnitud del cambio que producen en sus valores. Cuando un estado se actualiza significativamente, los estados predecesores que llevan a él se incorporan a la cola para ser actualizados después. De este modo, los cambios en el valor de un estado se propagan hacia atrás de manera más eficiente, evitando cálculos innecesarios y enfocándose en las partes del espacio de estados donde realmente importa. En ambientes determinísticos esto resulta especialmente potente, ya que cada acción tiene un único resultado y la propagación de los valores puede seguir trayectorias claras.

### ¿Qué es Trajectory Sampling?
Es una técnica que evita la necesidad de explorar todo el árbol de estados y acciones posibles durante el planeamiento. En lugar de un análisis exhaustivo, el método simula trayectorias completas o episodios ficticios siguiendo la política actual y va actualizando los valores de los estados y acciones visitados en ese recorrido.

### ¿Qué es Upper Confidence Bounds para Árboles (UCT)?
Es una estrategia de selección que resuelve el dilema entre exploración y explotación. Para cada acción, combina el valor promedio estimado de los resultados obtenidos hasta el momento con un término de confianza que favorece aquellas opciones que han sido probadas pocas veces. De esta manera, el algoritmo asegura que no se quede únicamente con las ramas que parecen mejores al inicio, sino que también explore alternativas menos visitadas que podrían resultar óptimas.

## Task 2. MCTS

In [1]:
import gymnasium as gym
import numpy as np

class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.children = {}  # Dictionary mapping actions to child nodes
        self.visits = 0
        self.total_reward = 0
        self.action = action # The action taken from the parent to reach this node

    def is_terminal(self, env):
        """Checks if the current state is a terminal state using information from env.unwrapped.desc."""
        unwrapped_env = env.unwrapped
        row, col = np.unravel_index(self.state, unwrapped_env.desc.shape)
        cell_type = unwrapped_env.desc[row, col].decode('utf-8')
        return cell_type in ['H', 'G']


    def is_fully_expanded(self, env):
        """Checks if all possible actions from this state have a corresponding child node."""
        num_actions = env.action_space.n
        return len(self.children) == num_actions

    def get_untried_actions(self, env):
        """Returns a list of actions that have not been tried from this node."""
        all_actions = list(range(env.action_space.n))
        tried_actions = self.children.keys()
        return [action for action in all_actions if action not in tried_actions]


class MCTS:
    """Manages the MCTS process."""
    def __init__(self, env, c_param=1.0):
        self.env = env
        initial_observation, info = self.env.reset()
        self.root = Node(initial_observation)  # Root node representing the initial state
        self.c_param = c_param

    def select(self, node):
        """Selects the best child node based on UCT."""
        while not node.is_terminal(self.env) and node.is_fully_expanded(self.env):
             node = self._best_child(node, self.c_param)
        return node

    def expand(self, node):
        """Adds a new child node for an untried action"""
        if not node.is_terminal(self.env) and not node.is_fully_expanded(self.env):
            untried_actions = node.get_untried_actions(self.env)
            if untried_actions:
                action = np.random.choice(untried_actions)
                unwrapped_env = self.env.unwrapped
                possible_outcomes = unwrapped_env.P[node.state][action]

                if possible_outcomes:
                    # Pick the first outcome for expansion.
                    probability, next_state, reward, done = possible_outcomes[0]
                    new_node = Node(next_state, parent=node, action=action)
                    node.children[action] = new_node
                    return new_node
        return node


    def simulate(self, node):
        """Performs a simulation from the given node's state using env.unwrapped.P."""
        current_state = node.state
        total_rollout_reward = 0
        done = node.is_terminal(self.env)

        temp_state = current_state
        unwrapped_env = self.env.unwrapped

        while not done:
            action = self.env.action_space.sample()

            possible_outcomes = unwrapped_env.P[temp_state][action]

            if not possible_outcomes:
                break

            outcomes_probs = [outcome[0] for outcome in possible_outcomes]
            if not np.isclose(sum(outcomes_probs), 1.0):
                 outcomes_probs = outcomes_probs / np.sum(outcomes_probs)

            chosen_outcome_index = np.random.choice(len(possible_outcomes), p=outcomes_probs)
            probability, next_state, reward, done_outcome = possible_outcomes[chosen_outcome_index]

            temp_state = next_state
            total_rollout_reward += reward

            if Node(temp_state).is_terminal(self.env):
                 done = True


        return total_rollout_reward


    def backpropagate(self, node, reward):
        """Backpropagates the reward up the tree."""
        current_node = node
        while current_node is not None:
            current_node.visits += 1
            current_node.total_reward += reward
            current_node = current_node.parent

    def _uct(self, node, c_param):
        """Calculates the UCT value for a child node."""
        if node.visits == 0:
            return float('inf')
        if node.parent is None or node.parent.visits == 0:
             return float('inf')

        return (node.total_reward / node.visits) + c_param * np.sqrt(np.log(node.parent.visits) / node.visits)

    def _best_child(self, node, c_param):
        """Selects the child with the highest UCT value."""
        best_child = None
        best_uct_value = -float('inf')
        for action, child in node.children.items():
            uct_value = self._uct(child, c_param)
            if uct_value > best_uct_value:
                best_uct_value = uct_value
                best_action = action
                best_child = child

        return best_child


    def run_mcts(self, num_simulations):
        """Runs the MCTS algorithm for a given number of simulations."""
        for _ in range(num_simulations):
            leaf_node = self.select(self.root)
            node_to_simulate_from = self.expand(leaf_node)
            rollout_reward = self.simulate(node_to_simulate_from)
            self.backpropagate(node_to_simulate_from, rollout_reward)

    def get_best_action(self):
        """Returns the action from the root node that leads to the child with the most visits."""
        if not self.root.children:
            return None

        best_action = None
        max_visits = -1
        for action, child_node in self.root.children.items():
            if child_node.visits > max_visits:
                max_visits = child_node.visits
                best_action = action
        return best_action

Running 5000 MCTS simulations...
MCTS simulations finished.
Recommended action from MCTS: 2
Taking action 2 in the environment.
Action taken: 2
Resulting state: 0
Reward: 0.0
Terminated: False
Truncated: False
Environment closed.


In [ ]:
import gymnasium as gym
import numpy as np

# Define the environment and MCTS parameters
env = gym.make("FrozenLake-v1", is_slippery=True)
num_episodes = 100
num_simulations_per_step = 5000

episode_rewards = []
successes = 0

for episode in range(num_episodes):
    observation, info = env.reset()
    mcts = MCTS(env, c_param=1.0)
    total_episode_reward = 0
    terminated = False
    truncated = False
    step_count = 0

    while not terminated and not truncated:
        mcts.root = Node(observation)
        mcts.run_mcts(num_simulations_per_step)
        action = mcts.get_best_action()
        next_observation, reward, terminated, truncated, info = env.step(action)
        total_episode_reward += reward
        observation = next_observation
        step_count += 1
    if reward > 0:
        successes += 1

    episode_rewards.append(total_episode_reward)
    if (episode + 1) % 10 == 0:
        print(f"Completed episode {episode + 1}/{num_episodes}")

env.close()

success_rate = (successes / num_episodes) * 100
average_reward = np.mean(episode_rewards)

print("\n--- Evaluation Results ---")
print(f"Success Rate: {success_rate:.2f}%")
print(f"Average Reward per Episode: {average_reward:.4f}")

Evaluating MCTS agent over 100 episodes with 5000 simulations per step.
Completed episode 10/100
Completed episode 20/100
Completed episode 30/100
